In [1]:
"""
TO DO LIST:
    - evaluate the default algorithm
        - holdout
        - confusion matrix
        - ROC / AUC
    
    - compare the default algorithm with the modified ones for single task (my dataset)
        - get each modified algorithm houldout accuracy and standard deviation (SD) estimates
        - calculate the difference in accuracy and SD between the default model and each of the mdified ones
        - Are the differences statistically significant? -> Run hypothesis tests (paired t-test or wilcoxon)

"""

'\nTO DO LIST:\n    - evaluate the default algorithm\n        - holdout\n        - confusion matrix\n        - ROC / AUC\n    \n    - compare the default algorithm with the modified ones for single task (my dataset)\n        - get each modified algorithm houldout accuracy and standard deviation (SD) estimates\n        - calculate the difference in accuracy and SD between the default model and each of the mdified ones\n        - Are the differences statistically significant? -> Run hypothesis tests (paired t-test or wilcoxon)\n\n'

In [12]:
# imports

import pandas as pd
import numpy as np
import openml
from AdaBoostWorkyWorky import AdaBoost
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
import scipy.stats as ss
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier

In [3]:
# get the data from OpenML
# dataset from Task ID = 3904

suite = openml.study.get_suite(99)
task_id = 3904
task = openml.tasks.get_task(task_id)    
dataset = openml.datasets.get_dataset(task.dataset_id)

X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="array", target=dataset.default_target_attribute
)

# Visualizar o dataset

df = pd.DataFrame(X, columns=attribute_names)
# converter:
#   0 -> -1
#   1 -> 1
df['target'] = 2*y-1 
df = df.dropna(how='any', axis=0)

X = df.drop(columns=['target'], axis=1)
y = df.target

df

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,target
0,1.1,1.4,1.4,1.4,1.3,1.300000,1.30,1.300000,1.300000,1.300000,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,-1
1,1.0,1.0,1.0,1.0,1.0,1.000000,1.00,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2,72.0,7.0,1.0,6.0,198.0,1134.130005,0.05,20.309999,55.849998,23029.099609,...,51.0,10.0,8.0,1.0,17.0,36.0,112.0,86.0,13.0,1
3,190.0,3.0,1.0,3.0,600.0,4348.759766,0.06,17.059999,254.869995,74202.671875,...,129.0,29.0,28.0,2.0,17.0,135.0,329.0,271.0,5.0,1
4,37.0,4.0,1.0,4.0,126.0,599.119995,0.06,17.190001,34.860001,10297.299805,...,28.0,1.0,6.0,0.0,11.0,16.0,76.0,50.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,18.0,4.0,1.0,4.0,52.0,241.479996,0.14,7.330000,32.930000,1770.859985,...,13.0,0.0,2.0,0.0,10.0,15.0,30.0,22.0,7.0,-1
10881,9.0,2.0,1.0,2.0,30.0,129.660004,0.12,8.250000,15.720000,1069.680054,...,5.0,0.0,2.0,0.0,12.0,8.0,19.0,11.0,3.0,-1
10882,42.0,4.0,1.0,2.0,103.0,519.570007,0.04,26.400000,19.680000,13716.719727,...,29.0,1.0,10.0,0.0,18.0,15.0,59.0,44.0,7.0,-1
10883,10.0,1.0,1.0,1.0,36.0,147.149994,0.12,8.440000,17.440001,1241.569946,...,6.0,0.0,2.0,0.0,9.0,8.0,21.0,15.0,1.0,-1


In [4]:
# holdout estimation function

def holdout_estimation(model, alpha, n_classifiers, x, y, test_size_value=0.3, seed=1111):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size_value, random_state=seed)
    model.fit(x_train, y_train, alpha, n_classifiers)
    y_pred = model.predict(x_test)
    #print(f"Estimated accuracy by holdout {accuracy_score(y_test, y_pred):.5f}, using {model.__class__.__name__}")
    return accuracy_score(y_test, y_pred)

In [5]:
# get the best number of classifiers in the AdaBoost for each alpha type of calc
best_n_interators = {}
for alpha in range(3):
    print("Calculating the best n for alpha =", alpha)
    best_n_interators[alpha] = (0, 0)
    for n in range(50, 151, 10):
        ab = AdaBoost()
        accuracy = holdout_estimation(ab, alpha, n, X, y)
        if (accuracy > best_n_interators[alpha][1]):
                best_n_interators[alpha] = (n, accuracy)

print(best_n_interators)

Calculating the best n for alpha = 0
Calculating the best n for alpha = 1
Calculating the best n for alpha = 2
{0: (50, 0.8183210784313726), 1: (140, 0.8186274509803921), 2: (120, 0.8131127450980392)}


In [6]:
# getting the accuracy results by holdout for 50 runs on the default and modified models

scores = {}
for alpha in range(3):
    scores[alpha] = []
    print("Alpha calc method =", alpha)
    for _ in range(50):
        ab = AdaBoost()
        scores[alpha].append(holdout_estimation(ab, alpha, best_n_interators[alpha][0], X, y))

for key in scores.keys():
    print(f"{key}: {scores.get(key)}")

Alpha calc method = 0
Alpha calc method = 1
Alpha calc method = 2
0: [0.8177083333333334, 0.8106617647058824, 0.8128063725490197, 0.8152573529411765, 0.8152573529411765, 0.8164828431372549, 0.8167892156862745, 0.8164828431372549, 0.8152573529411765, 0.8134191176470589, 0.8161764705882353, 0.805453431372549, 0.8106617647058824, 0.8174019607843137, 0.8164828431372549, 0.8146446078431373, 0.8149509803921569, 0.8164828431372549, 0.8118872549019608, 0.8155637254901961, 0.8152573529411765, 0.8121936274509803, 0.8152573529411765, 0.8164828431372549, 0.8125, 0.8186274509803921, 0.8088235294117647, 0.8177083333333334, 0.8174019607843137, 0.8143382352941176, 0.8183210784313726, 0.8121936274509803, 0.8146446078431373, 0.8161764705882353, 0.8164828431372549, 0.8183210784313726, 0.8149509803921569, 0.8149509803921569, 0.8134191176470589, 0.8100490196078431, 0.8137254901960784, 0.8164828431372549, 0.8174019607843137, 0.8180147058823529, 0.8177083333333334, 0.8155637254901961, 0.8155637254901961, 0.8

In [7]:
# check statistics for each model

averages = {}
stddev = {}
for alpha in range(3):
    averages[alpha] = np.mean(scores[alpha])
    stddev[alpha] = np.std(scores[alpha])

print("Averages for each method to calculate the alpha:\n\t", averages)
print()
print("Standard deviation for each method to calculate the alpha:\n\t", stddev)

Averages for each method to calculate the alpha:
	 {0: 0.8149632352941175, 1: 0.8170036764705884, 2: 0.7981678921568629}

Standard deviation for each method to calculate the alpha:
	 {0: 0.0026276407416110504, 1: 0.0016994703735733632, 2: 0.010066245771566638}


In [8]:
# differences between the results

default_scores = np.array(scores[0])
alpha1_scores = np.array(scores[1])
alpha2_scores = np.array(scores[2])

d_a1 = default_scores - alpha1_scores
d_a2 = default_scores - alpha2_scores
a1_a2 = alpha1_scores - alpha2_scores

print("Average of difference \t Standard Deviation of difference")
print(np.mean(d_a1), " \t", np.std(d_a1))
print(np.mean(d_a2), " \t", np.std(d_a2))
print(np.mean(a1_a2), " \t", np.std(a1_a2))

Average of difference 	 Standard Deviation of difference
-0.0020404411764705934  	 0.0026522804458316714
0.016795343137254913  	 0.01010930810699967
0.01883578431372551  	 0.009950127643844408


In [9]:
# compare models using paired t-test

t_test_d_a1 = ss.ttest_rel(default_scores, alpha1_scores) # --> returns tuple (statistic, p-value, df) --> we are mainly interested in the p-value
t_test_d_a2 = ss.ttest_rel(default_scores, alpha2_scores)
t_test_a1_a2 = ss.ttest_rel(alpha1_scores, alpha2_scores)

print("p-value for Default - Alpha1:", t_test_d_a1[1])
print("p-value for Default - Alpha2:", t_test_d_a2[1])
print("p-value for Alpha1 - Alpha2:", t_test_a1_a2[1])


p-value for Default - Alpha1: 2.0393834070912178e-06
p-value for Default - Alpha2: 1.0620902274728687e-15
p-value for Alpha1 - Alpha2: 8.060108727564165e-18


In [10]:
# compare models using wilcoxon signed rank test

wilcox_d_a1 = ss.wilcoxon(default_scores, alpha1_scores) # --> returns tuple (statistic, p-value) --> we are mainly interested in the p-value
wilcox_d_a2 = ss.wilcoxon(default_scores, alpha2_scores)
wilcox_a1_a2 = ss.wilcoxon(alpha1_scores, alpha2_scores)

print("p-value for Defalut - Alpha1:", wilcox_d_a1[1])
print("p-value for Defalut - Alpha2:", wilcox_d_a2[1])
print("p-value for Alpha1 - Alpha2:", wilcox_a1_a2[1])

p-value for Defalut - Alpha1: 5.7075242891466285e-06
p-value for Defalut - Alpha2: 1.7763568394002505e-15
p-value for Alpha1 - Alpha2: 1.7763568394002505e-15


d:\anaconda\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [13]:
models = [
    AdaBoost(0),
    AdaBoost(1),
    AdaBoost(2)
]

models

TypeError: __init__() takes 1 positional argument but 2 were given

In [11]:
# run cross-validation on an algorithm

def run_cv(X,y,algs,nfolds=10):
    results = {}
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=1111)
    for algo_name, algo in algs:
        results[algo_name] = []
        for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            algo.fit(X_train, y_train)
            y_pred = algo.predict(X_test)
            results[algo_name].append(accuracy_score(y_test, y_pred))
    results_df = pd.DataFrame.from_dict(results)
    return results_df


In [ ]:
# Plot the results using a box plot

def plot_cv(results_cv,metric='Accuracy'):
    fig, ax = plt.subplots()
    ax.boxplot(results_cv)
    ax.set_xticklabels(results_cv.columns)
    ax.set_ylabel(metric)
    ax.set_title("Cross-validation results for multiple algorithms in a single task")
    plt.show()

In [ ]:
# Plot the results using a bar plot with error bars

def plot_cv_estimates(results_cv,metric='Accuracy'):
    fig, ax = plt.subplots()
    ax.bar(results_cv.columns, results_cv.mean(), yerr=results_cv.std() / np.sqrt(results_cv.shape[0]), capsize=5)
    ax.set_ylabel(metric)
    ax.set_title("Cross-validation results for multiple algorithms in a single task")
    plt.show()